### Extracting daily SST and monthly heat flux fields for ACCESS-OM2-025 when using the ERA5 interannual forcing


In [1]:
#Load required packages
%matplotlib inline
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import cftime
from datetime import datetime

import cmocean as cm
import sys, os

from dask.distributed import Client
client = Client(processes=False)

In [2]:
# Load workers:
client = Client(n_workers=16)
client

/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33813 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/33813/status,
Dashboard: /proxy/33813/status,Workers: 16
Total threads: 32,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42991,Workers: 16
Dashboard: /proxy/33813/status,Total threads: 32
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:36501,Total threads: 2
Dashboard: /proxy/37555/status,Memory: 15.70 GiB
Nanny: tcp://127.0.0.1:38055,


In [3]:
%%time
base = ['/g/data/e14/rmh561/access-om2/archive/025deg_era5_iaf_1958cycle1/']

# reg = [-100, 20, 0, 60] # Ryan's North Atlantic region
reg = [-100, 20, 0, 70] # Zhi's North Atlantic region

for f in range(66):
    s = str('%03d' % (f,))
    year = 1958 + f
#     print('output'+str(s)+', '+str(year))
    if f % 5 == 0: print('----------------------------') # print a line every 5 output files

    if os.path.exists('/g/data/e14/mv7494/NASST_data/day_sst_fields_ERA5/day_NA_sst_fields_ERA5_'+str(year)+'.nc'): 
        print('output'+str(s)+' already done')
    else:
        print('creating data for output'+str(s))

        sst = xr.open_mfdataset(base[0] + 'output'+s+'/ocean/ocean_daily.nc',
                               chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'}).sst.sel(xt_ocean=slice(reg[0],reg[1]),yt_ocean=slice(reg[2],reg[3]))
        sst = sst.load()
        

        sst.attrs = {
                   'Project': 'North Atlantic Ocean Heat Wave 2023',
                   'Author': 'Maurice F. Huguenin', 
                   'Data set': 'ACCESS-OM2-025 IAF simulation (cycle6)',
                   'Unit': '(K)',
                   'Created': datetime.today().strftime('%Y-%m-%d'), 
                   'Mail': 'm.huguenin-virchaux@unsw.edu.au', 
                   'Funding': 'UNSW (Program code 1476), ARC ACEAS (ARC Grant No. SR200100008), ARC CLEX (ARC Grant No. CE170100023)'}
        # save as netcdf file
        sst.to_netcdf('/g/data/e14/mv7494/NASST_data/day_sst_fields_ERA5/day_NA_sst_fields_ERA5_'+str(year)+'.nc') # save as netcdf file

----------------------------
output000 already done
output001 already done
output002 already done
output003 already done
output004 already done
----------------------------
output005 already done
output006 already done
output007 already done
output008 already done
output009 already done
----------------------------
output010 already done
output011 already done
output012 already done
output013 already done
output014 already done
----------------------------
output015 already done
output016 already done
output017 already done
output018 already done
output019 already done
----------------------------
output020 already done
output021 already done
output022 already done
output023 already done
output024 already done
----------------------------
output025 already done
output026 already done
output027 already done
output028 already done
output029 already done
----------------------------
output030 already done
output031 already done
output032 already done
output033 already done
output034 alrea

In [4]:
### Next step, converting the heat flux terms to monthly values in (°C/s)

In [5]:
%%time
base = ['/g/data/e14/rmh561/access-om2/archive/025deg_era5_iaf_1958cycle1/']

# reg = [-100, 20, 0, 60] # Ryan's North Atlantic region
reg = [-100, 20, 0, 70]   # Zhi's North Atlantic region
variables = ['swflx','lw_heat', 'sens_heat', 'evap_heat', 'dht', 'pot_rho_0']

for f in range(66):
    s = str('%03d' % (f,))
    year = 1958 + f
#     print('output'+str(s)+', '+str(year))
    if f % 5 == 0: print('----------------------------') # print a line every 5 output files

    if os.path.exists('/g/data/e14/mv7494/NASST_data/mon_heat_flux_fields_ERA5/mon_NA_heat_flux_fields_ERA5_'+str(year)+'.nc'): 
        print('output'+str(s)+' already done')
    else:
        print('creating data for output'+str(s))

        ds = xr.open_mfdataset(base[0] + 'output'+s+'/ocean/ocean_month.nc',
                               chunks = {'yt_ocean': '200MB', 'xt_ocean': '200MB'})
        ds = ds[variables].sel(xt_ocean=slice(reg[0],reg[1]),yt_ocean=slice(reg[2],reg[3]))
        ds = ds.load()

        # Compute residual for check:
        ds['mld'] = ds['swflx'].copy()
        
        dzt = ds.dht; pot_rho_0 = ds.pot_rho_0 # allocate fields
        ds['mld'] = dzt.where(pot_rho_0 - pot_rho_0.isel(st_ocean=0) < 0.125).sum('st_ocean') # mixed layer depth

        ds.mld.attrs = {
            'long_name':'mixed layer depth (< 0.125 kg/m3 density difference from the surface)',
            'units': 'm',
            'cell_methods': 'time: mean',
            'standard_name': 'surface_net_downward_shortwave_flux'}
        ds.attrs = {
                   'Project': 'North Atlantic Ocean Heat Wave 2023',
                   'Author': 'Maurice F. Huguenin', 
                   'Data set': 'ACCESS-OM2-025 IAF simulation (cycle6)',
                   'Created': datetime.today().strftime('%Y-%m-%d'), 
                   'Mail': 'm.huguenin-virchaux@unsw.edu.au', 
                   'Funding': 'UNSW (Program code 1476), ARC ACEAS (ARC Grant No. SR200100008), ARC CLEX (ARC Grant No. CE170100023)'}
        # save as netcdf file
        ds.to_netcdf('/g/data/e14/mv7494/NASST_data/mon_heat_flux_fields_ERA5/mon_NA_heat_flux_fields_ERA5_'+str(year)+'.nc') # save as netcdf file

----------------------------
output000 already done
output001 already done
output002 already done
output003 already done
output004 already done
----------------------------
output005 already done
output006 already done
output007 already done
output008 already done
output009 already done
----------------------------
output010 already done
output011 already done
output012 already done
output013 already done
output014 already done
----------------------------
output015 already done
output016 already done
output017 already done
output018 already done
output019 already done
----------------------------
output020 already done
output021 already done
output022 already done
output023 already done
output024 already done
----------------------------
output025 already done
output026 already done
output027 already done
output028 already done
output029 already done
----------------------------
output030 already done
output031 already done
output032 already done
output033 already done
output034 alrea